In [54]:
'''Plan:
1. Load libraries, load data.
2. Preliminary EDA, dealing with missing values, merging train and test.
3. EDA, deleting variables.
4. Feature engineering, ohc.
5. Sample formation.
6. Feature scaling.
7. Model fitting.
8. Performance evaluation.
9. Predictions.
'''

### 1. Load libraries ###

import numpy as np
import pandas as pd
import os, warnings, random, time
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer

from xgboost import XGBClassifier

warnings.filterwarnings("ignore")
#os.getcwd()

In [55]:
### Load data ###

train = pd.read_csv('../input/tabular-playground-series-sep-2021/train.csv')
print(train.shape)

test = pd.read_csv('../input/tabular-playground-series-sep-2021/test.csv')

(957919, 120)


In [56]:
train = train.sample(n=50000)

In [57]:
### 2. ###

train.shape
train.describe()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
count,50000.000000,49239.000000,49204.000000,49156.000000,49172.000000,49188.000000,49156.000000,49173.000000,4.918400e+04,4.923200e+04,...,49247.000000,49246.000000,49239.000000,49173.000000,49159.000000,49195.000000,4.915600e+04,49188.000000,49188.000000,50000.000000
mean,478811.773080,0.090148,0.345875,4083.993706,0.201427,0.304232,-0.078651,1605.932553,3.740732e+05,1.824343e+15,...,-19.879890,2.078207,23.570016,1.700365,63866.922886,1.208409,4.230062e+16,3964.214780,0.559079,0.499480
std,276869.735891,0.043462,0.145976,6453.816083,0.212285,0.146260,2.102980,1274.775041,3.440792e+05,2.345513e+15,...,18.498894,0.898091,45.278472,10.108747,92835.579408,0.115461,6.688385e+16,3160.828858,0.406363,0.500005
min,18.000000,-0.128380,-0.018642,-8370.600000,-0.053539,-0.006659,-12.097000,-198.590000,-2.421600e+04,-1.050800e+15,...,-102.410000,0.305350,-24.187000,-26.346000,-54197.000000,0.905270,-6.779000e+15,-407.920000,-0.122870,0.000000
25%,238495.750000,0.070059,0.282540,420.412500,0.034772,0.239080,-1.113400,460.170000,8.966325e+04,1.164225e+13,...,-28.657500,1.489000,-0.616865,-4.549200,2499.500000,1.146500,2.248250e+14,1307.725000,0.276500,0.000000
50%,477330.500000,0.090157,0.388850,1287.250000,0.138080,0.327085,-0.401460,1426.500000,2.868950e+05,5.247250e+14,...,-14.660000,1.662900,1.724000,0.807710,20131.000000,1.176800,1.305300e+16,3226.200000,0.474350,0.000000
75%,719372.250000,0.116830,0.458410,4396.950000,0.297975,0.413793,0.901095,2483.600000,5.551250e+05,3.157325e+15,...,-5.373950,2.546550,18.405500,6.835800,90891.500000,1.240400,5.219100e+16,6150.800000,0.747753,1.000000
max,957913.000000,0.410040,0.518990,39544.000000,1.285600,0.549120,10.550000,5337.200000,1.871000e+06,1.030400e+16,...,1.399100,4.506600,211.930000,46.983000,511400.000000,1.789600,3.217600e+17,13148.000000,2.632000,1.000000


In [58]:
# are there numerical features?

un_colval = pd.DataFrame([[x,len(train[x].unique())] for x in train.columns], columns = ['colname', 'n_unique'])
un_colval.loc[un_colval.n_unique < 100]
# all columns contain numerical features

,colname,n_unique
119,claim,2


In [59]:
train.describe()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
count,50000.000000,49239.000000,49204.000000,49156.000000,49172.000000,49188.000000,49156.000000,49173.000000,4.918400e+04,4.923200e+04,...,49247.000000,49246.000000,49239.000000,49173.000000,49159.000000,49195.000000,4.915600e+04,49188.000000,49188.000000,50000.000000
mean,478811.773080,0.090148,0.345875,4083.993706,0.201427,0.304232,-0.078651,1605.932553,3.740732e+05,1.824343e+15,...,-19.879890,2.078207,23.570016,1.700365,63866.922886,1.208409,4.230062e+16,3964.214780,0.559079,0.499480
std,276869.735891,0.043462,0.145976,6453.816083,0.212285,0.146260,2.102980,1274.775041,3.440792e+05,2.345513e+15,...,18.498894,0.898091,45.278472,10.108747,92835.579408,0.115461,6.688385e+16,3160.828858,0.406363,0.500005
min,18.000000,-0.128380,-0.018642,-8370.600000,-0.053539,-0.006659,-12.097000,-198.590000,-2.421600e+04,-1.050800e+15,...,-102.410000,0.305350,-24.187000,-26.346000,-54197.000000,0.905270,-6.779000e+15,-407.920000,-0.122870,0.000000
25%,238495.750000,0.070059,0.282540,420.412500,0.034772,0.239080,-1.113400,460.170000,8.966325e+04,1.164225e+13,...,-28.657500,1.489000,-0.616865,-4.549200,2499.500000,1.146500,2.248250e+14,1307.725000,0.276500,0.000000
50%,477330.500000,0.090157,0.388850,1287.250000,0.138080,0.327085,-0.401460,1426.500000,2.868950e+05,5.247250e+14,...,-14.660000,1.662900,1.724000,0.807710,20131.000000,1.176800,1.305300e+16,3226.200000,0.474350,0.000000
75%,719372.250000,0.116830,0.458410,4396.950000,0.297975,0.413793,0.901095,2483.600000,5.551250e+05,3.157325e+15,...,-5.373950,2.546550,18.405500,6.835800,90891.500000,1.240400,5.219100e+16,6150.800000,0.747753,1.000000
max,957913.000000,0.410040,0.518990,39544.000000,1.285600,0.549120,10.550000,5337.200000,1.871000e+06,1.030400e+16,...,1.399100,4.506600,211.930000,46.983000,511400.000000,1.789600,3.217600e+17,13148.000000,2.632000,1.000000


In [60]:
test.describe()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
count,4.934740e+05,485662.000000,485583.000000,485679.000000,485741.000000,485597.000000,485561.000000,485541.000000,4.856190e+05,4.856560e+05,...,485461.000000,485701.000000,485585.000000,485449.000000,485718.000000,485532.000000,485497.000000,4.853910e+05,485711.000000,485589.000000
mean,1.204656e+06,0.090191,0.346143,4047.202400,0.201503,0.304879,-0.071511,1617.036351,3.767998e+05,1.808500e+15,...,0.398130,-19.840970,2.075146,23.846392,1.762625,63051.287541,1.209049,4.284258e+16,3968.585512,0.558734
std,1.424538e+05,0.043538,0.145889,6383.224961,0.212553,0.145435,2.125074,1272.769177,3.450146e+05,2.337721e+15,...,0.297971,18.513063,0.895702,45.544851,10.096886,92279.031878,0.115074,6.743966e+16,3161.273268,0.408472
min,9.579190e+05,-0.153190,-0.019060,-9436.300000,-0.080716,-0.007007,-12.841000,-215.660000,-2.780900e+04,-1.254900e+15,...,-0.042181,-104.740000,0.261040,-23.938000,-26.311000,-81381.000000,0.900700,-7.749400e+15,-416.240000,-0.186960
25%,1.081287e+06,0.070211,0.283660,418.835000,0.035169,0.240600,-1.119500,481.820000,9.142400e+04,1.158000e+13,...,0.113300,-28.625000,1.488500,-0.624700,-4.476575,2474.875000,1.146800,2.317850e+14,1310.000000,0.276760
50%,1.204656e+06,0.090173,0.388940,1277.900000,0.137350,0.328020,-0.380190,1441.800000,2.895900e+05,5.048250e+14,...,0.357960,-14.576000,1.663100,1.730400,0.910985,19429.000000,1.177200,1.330400e+16,3237.100000,0.473790
75%,1.328024e+06,0.116507,0.458310,4421.400000,0.297160,0.412810,0.923240,2490.000000,5.596300e+05,3.106600e+15,...,0.622650,-5.318300,2.524700,18.960000,6.852075,88309.500000,1.242300,5.284700e+16,6157.300000,0.744970
max,1.451392e+06,0.412900,0.519880,39841.000000,1.310200,0.555580,11.046000,5413.100000,1.906700e+06,1.048900e+16,...,1.109400,1.464200,4.570500,217.340000,47.314000,521540.000000,1.877900,3.239500e+17,13198.000000,2.732000


In [61]:
# imputing missing values #

colmnames = train.columns
test['claim'] = np.nan
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(train)
train = pd.DataFrame(imp.transform(train))
test = pd.DataFrame(imp.transform(test))
#train.describe()

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,5.000000e+04,5.000000e+04,...,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,50000.000000
mean,478811.773080,0.090148,0.346559,4036.784673,0.200378,0.304603,-0.084100,1602.964739,3.726504e+05,1.804381e+15,...,-19.801278,2.071944,23.237519,1.685601,63131.284663,1.207900,4.180692e+16,3952.229419,0.557703,0.499480
std,276869.735891,0.043130,0.144909,6409.247385,0.210675,0.145097,2.085570,1264.395604,3.414386e+05,2.332911e+15,...,18.370070,0.892729,45.012105,10.025443,92223.169262,0.114597,6.642388e+16,3136.444882,0.403192,0.500005
min,18.000000,-0.128380,-0.018642,-8370.600000,-0.053539,-0.006659,-12.097000,-198.590000,-2.421600e+04,-1.050800e+15,...,-102.410000,0.305350,-24.187000,-26.346000,-54197.000000,0.905270,-6.779000e+15,-407.920000,-0.122870,0.000000
25%,238495.750000,0.070348,0.285290,431.227500,0.035847,0.241050,-1.102725,474.875000,9.199000e+04,1.251800e+13,...,-28.349250,1.491800,-0.582995,-4.433400,2666.325000,1.147000,2.384725e+14,1337.725000,0.280527,0.000000
50%,477330.500000,0.090157,0.388850,1287.250000,0.138080,0.327085,-0.401460,1426.500000,2.868950e+05,5.247250e+14,...,-14.660000,1.662900,1.724000,0.807710,20131.000000,1.176800,1.305300e+16,3226.200000,0.474350,0.000000
75%,719372.250000,0.115913,0.457470,4273.275000,0.294432,0.412253,0.885263,2462.225000,5.487450e+05,3.100550e+15,...,-5.481025,2.500000,17.863750,6.699800,88386.000000,1.238600,5.098750e+16,6098.625000,0.741663,1.000000
max,957913.000000,0.410040,0.518990,39544.000000,1.285600,0.549120,10.550000,5337.200000,1.871000e+06,1.030400e+16,...,1.399100,4.506600,211.930000,46.983000,511400.000000,1.789600,3.217600e+17,13148.000000,2.632000,1.000000


In [63]:
train['sample'] = 'train'
test['sample'] = 'pred'
df = pd.concat([train, test])
df.colnames = colmnames
df

,0,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,sample
0,287817.0,0.082316,0.487200,851.79,0.498290,0.378830,1.599000,1924.500,56523.0,3.523800e+15,...,1.8735,-3.50130,18.75400,106310.000,1.0817,5.456700e+16,356.00,0.327350,1.0,train
1,780501.0,0.097944,0.416790,4116.90,0.096623,0.492870,-1.549200,3877.100,612420.0,5.843300e+14,...,1.5398,156.46000,-1.51470,-30.191,1.1788,4.072500e+15,3332.00,0.040974,1.0,train
2,278317.0,0.100590,0.453950,26235.00,0.411310,0.419510,2.247100,4052.000,317950.0,9.195500e+15,...,1.4981,1.99030,1.53370,97295.000,1.2842,2.276800e+17,2285.10,0.416590,1.0,train
3,483674.0,0.077372,0.257680,420.55,0.006061,0.332850,-0.122640,2783.000,553000.0,3.082900e+15,...,3.8728,14.59200,0.59872,107120.000,1.5243,1.549000e+16,1915.00,0.662360,1.0,train
4,836873.0,0.132260,0.005187,431.05,0.004482,0.000921,-1.118500,75.313,312930.0,3.960700e+15,...,4.0366,-1.43330,-8.96360,-9278.200,1.1478,2.028100e+16,1144.40,0.501480,0.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493469,1451388.0,-0.009112,0.308190,637.64,0.778200,0.414150,-1.068500,651.220,985000.0,6.079700e+15,...,2.3325,0.11226,-5.92380,20131.000,1.1559,8.163900e+16,857.09,1.563300,0.0,pred
493470,1451389.0,0.088922,0.482650,6924.10,0.025963,0.355400,-0.870200,2514.200,18004.0,6.073500e+14,...,1.7005,97.81300,4.37930,-2432.000,1.0707,4.691800e+16,7497.10,0.670750,0.0,pred
493471,1451390.0,0.140620,0.484750,1797.10,0.147020,0.288030,-1.407100,434.030,333050.0,2.351000e+15,...,1.6827,1.72400,8.06330,2471.400,1.1725,7.900900e+16,2904.60,0.180050,0.0,pred
493472,1451391.0,0.168000,0.351760,454.79,0.164580,0.169830,0.323850,2331.200,223980.0,-2.795300e+12,...,1.3531,155.21000,13.96300,-11.440,1.1946,-1.770600e+14,6763.10,0.332230,0.0,pred


In [28]:
### 3. ###

# we have many variables, are all of them useful? lets see correlation

cors = df.corr()
cors.loc[((cors > 0.8)&(cors<1)).any(1)]
# there are no pairwise correlations above 80%

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim


In [29]:
### 4. ###

# check skew and possibly transform some variables #


In [30]:
### 5. ###

y_train = df[['claim']]
X_train = df.drop(columns=['claim'])

print(y_train.shape, X_train.shape)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=2)

print(y_train.shape, X_train.shape, X_test.shape)

(37423, 1) (37423, 119)
(33680, 1) (33680, 119) (3743, 119)


In [31]:
### 6. Scaling ###

ss = StandardScaler()

for i in X_train.columns:
    X_train[[i]] = ss.fit_transform(X_train[[i]])
    X_test[[i]] = ss.transform(X_test[[i]])

X_train.describe()  

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
count,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,...,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04,3.368000e+04
mean,-4.614941e-17,-1.723367e-16,-2.276847e-16,-6.709136e-17,6.759900e-17,-2.844782e-18,-6.526846e-19,-8.897607e-17,-1.684585e-16,1.585166e-16,...,3.469117e-17,4.014999e-17,4.683259e-16,2.453995e-17,1.165602e-17,1.413491e-17,-1.023653e-15,5.385595e-17,4.349912e-17,5.301249e-17
std,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,...,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00,1.000015e+00
min,-1.735789e+00,-5.377914e+00,-2.483372e+00,-1.994204e+00,-1.250938e+00,-2.163310e+00,-5.914170e+00,-1.425799e+00,-1.166974e+00,-1.223646e+00,...,-1.450151e+00,-4.443746e+00,-1.926185e+00,-1.131163e+00,-2.738443e+00,-1.138977e+00,-2.559208e+00,-7.435901e-01,-1.380015e+00,-1.657057e+00
25%,-8.660363e-01,-4.612403e-01,-4.195743e-01,-5.645006e-01,-7.877872e-01,-4.352402e-01,-4.988670e-01,-8.925065e-01,-8.301976e-01,-7.676110e-01,...,-9.606197e-01,-4.766457e-01,-6.542054e-01,-5.381229e-01,-6.175693e-01,-6.511689e-01,-5.308956e-01,-6.312502e-01,-8.443441e-01,-6.910875e-01
50%,4.651390e-03,-7.531020e-03,2.968736e-01,-4.295383e-01,-2.977853e-01,1.566639e-01,-1.507298e-01,-1.215875e-01,-2.466854e-01,-5.550956e-01,...,-1.098074e-01,2.800509e-01,-4.597107e-01,-4.861041e-01,-8.722818e-02,-4.740574e-01,-2.710345e-01,-4.371232e-01,-2.344897e-01,-2.052726e-01
75%,8.684124e-01,5.965637e-01,7.664012e-01,5.870339e-02,4.611632e-01,7.397065e-01,4.727733e-01,6.836539e-01,5.259617e-01,5.483510e-01,...,7.524793e-01,7.880920e-01,4.845858e-01,-1.085895e-01,5.034799e-01,2.646904e-01,2.783364e-01,1.428407e-01,6.975473e-01,4.561320e-01
max,1.733535e+00,7.155931e+00,1.162921e+00,5.495634e+00,4.972745e+00,1.666740e+00,5.136039e+00,2.962957e+00,4.372166e+00,3.658337e+00,...,2.391472e+00,1.139968e+00,2.794379e+00,4.063584e+00,4.346456e+00,4.770453e+00,5.461607e+00,4.170953e+00,2.891493e+00,5.204625e+00


In [32]:
### 7. Model fitting ###

time1 = time.time()

log_rg = LogisticRegression()

grid_values = {'penalty': ['l2'], 'C': [0.00001, 0.0001, 0.001, 0.01]}

lr = GridSearchCV(log_rg, param_grid = grid_values, cv=4)
lr.fit(X_train, y_train)

print('logistic', lr.best_score_, lr.best_params_, time.time()-time1)


logistic 0.8619358669833729 {'C': 1e-05, 'penalty': 'l2'} 2.049663782119751


In [34]:
time1 = time.time()

svm = svm.SVC(kernel='rbf')

grid_values = {'C':[0.01, 0.1, 1, 5, 10]}

svm = GridSearchCV(svm, param_grid = grid_values, cv=2)
svm.fit(X_train, y_train)

print('SVM', svm.best_score_, svm.best_params_, time.time()-time1)

KeyboardInterrupt: 

In [33]:
time1 = time.time()

xgbcl = XGBClassifier()

grid_values = {'eta':[0.01, 0.1], 'max_depth':[2,3,5]}

xgb = GridSearchCV(xgbcl, param_grid = grid_values, cv=2)
xgb.fit(X_train, y_train)

print('XGBoost', xgb.best_score_, xgb.best_params_, time.time()-time1)

[18:26:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:26:33] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:26:44] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[18:27:01] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

KeyboardInterrupt: 